In [6]:
sets = [
    {"size":10, "instances": ["5", "6b", "7"]},
    {"size":15, "instances": ["14"]},
    {"size":20, "instances": ["7", "12", "17"]},
    {"size":30, "instances": ["10", "20"]},
    #{"size":40, "instances": ["8", "15", "25"]}

    #{"size":20, "instances": ["7"]},
    #{"size":30, "instances": ["10"]},
    #{"size":40, "instances": ["8"]}
]

In [7]:
for set in sets:
    
    size = set["size"]
    
    for instance in set["instances"]:
        
        !echo "./IN/graf_{size}_{instance}.txt"
        !cmake-build-debug/pdp --source "./IN/graf_{size}_{instance}.txt" --threads 8

./IN/graf_10_5.txt
total time: 0.000303719
./IN/graf_10_6b.txt
total time: 0.00149665
./IN/graf_10_7.txt
total time: 0.000356464
./IN/graf_15_14.txt
total time: 0.00160239
./IN/graf_20_7.txt
total time: 0.00811867
./IN/graf_20_12.txt
total time: 0.0251161
./IN/graf_20_17.txt
total time: 0.0300382
./IN/graf_30_10.txt
total time: 7.01253
./IN/graf_30_20.txt
total time: 19.4127
